# CT Scan Image Reconstruction: ART vs HHL

## Author: Nischal Gautam

### Description: This notebook demonstrates tomographic reconstruction of a simple 2×2 phantom using Algebraic Reconstruction Technique (ART) and the Quantum Harrow-Hassidim-Lloyd (HHL) algorithm implemented via Qiskit.

Imports

In [1]:
# Copyright (c) 2025 Nischal Gautam

import numpy as np
import matplotlib.pyplot as plt
import math
from math import pi, asin
from scipy.linalg import expm
from skimage.transform import radon
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector, Operator


Generate Phantom and  Sinogram

In [ ]:
# Create a simple 2×2 phantom image
phantom2 = np.array([[1, 2],
                     [3, 4]], float)

# Compute sinogram using Radon transform from 0° to 180°
angles   = np.arange(0, 181, 1)
sinogram = radon(phantom2, theta=angles, circle=False)

# Plot the sinogram
plt.figure(figsize=(4, 3))
plt.imshow(sinogram,
           extent=(angles.min(), angles.max(), 0, sinogram.shape[0]),
           cmap='gray', aspect='auto')
plt.title('Sinogram of 2x2 Phantom')
plt.xlabel('Projection Angle (°)')
plt.ylabel('Detector Position')
plt.tight_layout()
plt.show()


Build Projection Matrix and Measurement Vector

In [ ]:
# Define pixel coordinates for 2x2 image
coords = np.array([[-0.5, -0.5],
                   [ 0.5, -0.5],
                   [-0.5,  0.5],
                   [ 0.5,  0.5]])

# Build projection matrix A4 and measurement vector b4 for 0° and 90°
A4, b4 = [], []
for phi_deg in [0, 90]:
    phi = math.radians(phi_deg)
    for det in range(2):
        d = det - 0.5
        row, val = [], 0.0
        for pix, (x, y) in enumerate(coords):
            hit = float(abs(d - (x*math.cos(phi) + y*math.sin(phi))) < 1e-6)
            row.append(hit)
            val += hit * phantom2.flatten()[pix]
        A4.append(row)
        b4.append(val)
A4 = np.array(A4)
b4 = np.array(b4)


Algebraic Reconstruction Technique (ART)

In [ ]:
# Simple ART solver for Ax = b
iters, relax = 10, 1.0
x_art = np.zeros(4)
for _ in range(iters):
    for i in range(4):
        ai = A4[i]
        n2 = ai.dot(ai)
        if n2 > 0:
            r = b4[i] - ai.dot(x_art)
            x_art += relax * (r / n2) * ai
art_recon = x_art.reshape(2, 2)


Setup for HHL Algorithm

In [ ]:
# Hermitian matrix and normalized vector for HHL
A_herm = A4.T @ A4 + np.eye(4)
b_herm = A4.T @ b4
b_norm = b_herm / np.linalg.norm(b_herm)

# Time scaling factor from max eigenvalue
λ_max = max(np.linalg.eigvals(A_herm).real)
t0    = 2 * pi / λ_max

# Precompute U = exp(i*A*t) for QPE
U_ops = [
    Operator(expm(1j * A_herm * (2**k) * t0)).to_instruction().control(1)
    for k in range(5)
]


Define QFT and QPE Subroutines

In [ ]:
# Quantum registers
anc   = QuantumRegister(1, 'anc')
clk   = QuantumRegister(5, 'clk')   # 5 clock qubits
sysq  = QuantumRegister(2, 'sys')
creg  = ClassicalRegister(3, 'c')
qc    = QuantumCircuit(anc, clk, sysq, creg)

# QFT (Quantum Fourier Transform)
def qft(circ):
    circ.barrier(label="QFT-start")
    for i in range(5 // 2):
        circ.swap(clk[i], clk[4 - i])
    for j in range(5):
        circ.h(clk[j])
        for k in range(j+1, 5):
            circ.cp(pi / 2**(k-j), clk[k], clk[j])
    circ.barrier(label="QFT-end")

# Inverse QFT
def qft_dg(circ):
    circ.barrier(label="iQFT-start")
    for j in reversed(range(5)):
        for k in reversed(range(j+1, 5)):
            circ.cp(-pi / 2**(k-j), clk[k], clk[j])
        circ.h(clk[j])
    for i in range(5 // 2):
        circ.swap(clk[i], clk[4 - i])
    circ.barrier(label="iQFT-end")


Define QPE, and InvQPE Subroutines

In [ ]:
# Quantum Phase Estimation
def qpe(circ):
    circ.barrier(label="QPE-start")
    circ.h(clk)
    circ.barrier(label="QPE-H")
    for k in range(5):
        circ.append(U_ops[k], [clk[k], *sysq])
    circ.barrier(label="QPE-U")
    qft_dg(circ)
    circ.barrier(label="QPE-end")

# Inverse QPE
def inv_qpe(circ):
    circ.barrier(label="InvQPE-start")
    qft(circ)
    for k in reversed(range(5)):
        circ.append(U_ops[k].inverse(), [clk[k], *sysq])
    circ.barrier(label="InvQPE-end")



Define HHL Subroutine

In [ ]:
# HHL Subroutine
def hhl(circ):
    circ.barrier(label="HHL-init")
    circ.initialize(b_norm, sysq)
    qpe(circ)
    circ.barrier(label="Rot-start")
    # Eigenvalue inverse encoding (example: simplified for 3 eigenvalues)
    phase_to_lambda = {0b00110:1, 0b10011:3, 0b00000:5}
    for pat, lam in phase_to_lambda.items():
        theta = 2 * asin(1/lam)
        bits  = format(pat, '05b')
        for i, bit in enumerate(reversed(bits)):
            if bit == '0':
                circ.x(clk[i])
        circ.mcry(theta, clk, anc[0])
        for i, bit in enumerate(reversed(bits)):
            if bit == '0':
                circ.x(clk[i])
    circ.barrier(label="Rot-end")
    inv_qpe(circ)
    circ.measure(anc, creg[0])
    circ.barrier(label="HHL-complete")

Build and Visualize Quantum Circuit

In [ ]:
qc.barrier(label="Start")
hhl(qc)
qc.measure(sysq[0], creg[1])
qc.measure(sysq[1], creg[2])
qc.barrier(label="End")

# Draw circuit
qc.draw('mpl', fold=33)


Postselect & Extract HHL Solution

In [ ]:
# Remove final measurements for simulation
qc_nom = qc.remove_final_measurements(inplace=False)

# Simulate the quantum circuit
state  = Statevector.from_instruction(qc_nom)

# Reshape to access subsystem: ancilla • clk0–4 • sys0 • sys1
tensor = state.data.reshape([2]*8)

# Post-select on ancilla=1, marginalize over clock qubits
sys_amps = tensor[1].sum(axis=(0,1,2,3,4))

# Normalize and scale the output
vec = sys_amps.flatten()
vec /= np.linalg.norm(vec)
vec *= (1.0 / vec[0].real)

# Reorder amplitudes and reshape
x_qhl = vec[[3,1,2,0]].reshape(2,2)
hhl_recon = np.real(x_qhl)


Plot Final Reconstructions

In [ ]:
# Show phantom, ART, and HHL reconstructions
fig, axes = plt.subplots(1, 3, figsize=(9, 3))
axes[0].imshow(phantom2,   cmap='gray'); axes[0].set_title('Original Phantom');    axes[0].axis('off')
axes[1].imshow(art_recon,  cmap='gray'); axes[1].set_title('ART Reconstruction');  axes[1].axis('off')
axes[2].imshow(hhl_recon,  cmap='gray'); axes[2].set_title('HHL Reconstruction');  axes[2].axis('off')
plt.tight_layout()
plt.show()
